# Test model performance

In this notebook, we are going to load in some new data and test how well our ensemble of random forest models predicts if users will own certain games. 

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pickle
import requests
import random
import re
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn import metrics
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
import ast
from gensim import corpora, models, similarities
from nltk.tokenize import word_tokenize
import operator
import itertools

%matplotlib inline
import matplotlib.pyplot as plt

# Define SQL database info
db_name  = 'UserInfo'
username = 'username'
host     = 'localhost'
pwd      = 'password'
port     = '5432'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(username, pwd, host, port, db_name))
print(engine.url)

# connect to database:
con = None
con = psycopg2.connect(database = db_name, user = username, password = pwd, host = host)
cur = con.cursor() # get a cursor to our current connection

## Load in user data

In [ ]:
numUsers = 1000 # Number of users to train the model
# make a query to initialize the allPlayers table
create_table_sql = """
SELECT * FROM allPlayers WHERE gameCount IS NOT NULL ORDER BY random() LIMIT {};
""".format(numUsers)
userFrame = pd.read_sql_query(create_table_sql,con)

# For now we have to remove users that have the same game information (earlier error in writing data to database)
userFrame = userFrame.drop_duplicates(subset='games',keep="last")
len(userFrame)

## Load in TF-IDF corpus and dictionary

This will be used to compute a similarity score between games to be tested, and the games that each user currently owns.

In [ ]:
# Load in the pretrained tf-idf models,dictionary, and corpus for game tags and game descriptions
tag_dictionary = corpora.Dictionary.load('/home/gameTagDict.dict')
tag_corpus     = corpora.MmCorpus('/home/gameTagCorpus.mm')
description_dictionary = corpora.Dictionary.load('/home/gameDescriptionDict.dict')
description_corpus     = corpora.MmCorpus('/home/gameDescriptionCorpus.mm')

# Initialise a term-frequency inverse document frequency model based on the corpus for both tags and descriptions
tag_tfidf = models.TfidfModel(tag_corpus) 
description_tfidf =  models.TfidfModel(description_corpus)
# Initialise a similarity index using the stored index, corpus, and dictionary
tag_sims = similarities.Similarity('/home/',
                                   tag_tfidf[tag_corpus],num_features=len(tag_dictionary))
description_sims = similarities.Similarity('/home/',
                    description_tfidf[description_corpus],num_features=len(description_dictionary))

## Load in game data

In [ ]:
# Now let's load in the game data
# make a query to initialize the allGames table
create_table_sql = """
SELECT * FROM allGames WHERE gamename IS NOT NULL;
"""
gameFrame = pd.read_sql_query(create_table_sql,con)
gameFrame = gameFrame.dropna(axis=0, how='any') # Remove any rows that have nans
len(gameFrame)

## Define functions to compute similarity of game descriptions and tags

In [ ]:
allGames = gameFrame.gameid.tolist()
def compute_tag_similarity(testGame,userGameDict,ind):
    # This function comutes the similarity of one game to the games a user already owns based on game tags
    testRow     = gameFrame[gameFrame['gameid']==testGame]
    tmp         = testRow.tags.tolist()
    currentTags = ast.literal_eval(tmp[0]) # Evaluate to get in list form
    query       = ' '.join(currentTags) # Join all together with spaces
    print("Computing tag similarity for user {} and game {}".format(ind,testGame),end='\r')
    
    # Tokenize
    query_doc = [w.lower() for w in word_tokenize(query)]
    # Compare to dictionary we computed on all other games
    query_doc_bow = tag_dictionary.doc2bow(query_doc)
    # Vectorize word representation
    query_doc_tf_idf = tag_tfidf[query_doc_bow]
    # Compute the similarity of the game with current tags to al games in our database
    query_similarity = tag_sims[query_doc_tf_idf]

    # Control for if the user has only one game
    if type(userGameDict) == dict:
        userGameDict = list([userGameDict])
        
    gamesOwned  = []
    gamesPlayed = []
    for g in userGameDict:
        if g['appid'] in allGames:
            if g['appid'] != testGame: # If the user owns the testGame, we should omit this from the model
                gamesOwned.append(g['appid'])
                gamesPlayed.append(g['playtime_forever']+0.01) # Add small amount to avoid dividing by zero
    # normalize the playtime to the total amount of playtime
    gamesPlayed = np.array([x / sum(gamesPlayed) for x in gamesPlayed]) 
    
    # If the user owns the testGame, we should omit this from the model
    if testGame in gamesOwned:
        gamesOwned.remove(testGame)
    
    # We only want to compare to games that are in our original list
    testInd = []
    for game in gamesOwned:
        if game in allGames:
            testInd.append(allGames.index(game))
    user2game_similarity = query_similarity[testInd]
    user2game_weighted_similarity = user2game_similarity*gamesPlayed
    
    # If we just have empty arrays, then fill with 0
    if user2game_similarity.size < 1:
        user2game_similarity = 0
        user2game_weighted_similarity = 0
        
    return np.median(user2game_similarity)#, np.median(user2game_weighted_similarity)

def preprocess_descriptions(description):
    # Define a list of words that we will not include, since they do not convey meaning
    try:
        removelist = set('for a of the and to in on is or be as where it its at an - with by'.split())
        description = re.sub(r"\s+", " ", description) # remove line breaks and tabs
        description = re.sub(r"<.*?>", "", description) # remove <>
        description = description.replace(",","") # remove commas
        description = description.replace(".","") # remove periods
        # Split text into a list of words
        text = [word for word in description.lower().split() if word not in removelist]
        return text
    except:
        return []

def compute_description_similarity(testGame,userGameDict,ind):
    # This function comutes the similarity of one game to the games a user already owns based on game description
    testRow     = gameFrame[gameFrame['gameid']==testGame]
    tmp         = testRow.detailedescription.tolist()
    query_doc   = preprocess_descriptions(tmp[0]) # preprocess data
    print("Computing description similarity for user {} and game {}".format(ind,testGame),end='\r')
    # Compare to dictionary we computed on all other games
    query_doc_bow = description_dictionary.doc2bow(query_doc)
    # Vectorize word representation
    query_doc_tf_idf = description_tfidf[query_doc_bow]
    # Compute the similarity of the game with current tags to al games in our database
    query_similarity = description_sims[query_doc_tf_idf]
    
    # Control for if the user has only one game
    if type(userGameDict) == dict:
        userGameDict = list([userGameDict])
    
    gamesOwned  = []
    gamesPlayed = []
    for g in userGameDict:
        if g['appid'] in allGames:
            if g['appid'] != testGame: # If the user owns the testGame, we should omit this from the model
                gamesOwned.append(g['appid'])
                gamesPlayed.append(g['playtime_forever']+0.01)
    # normalize the playtime to the total amount of playtime
    gamesPlayed = np.array([x / sum(gamesPlayed) for x in gamesPlayed]) 
    
    # We only want to compare to games that are in our original list
    testInd = []
    for game in gamesOwned:
        if game in allGames:
            testInd.append(allGames.index(game))
    user2game_similarity = query_similarity[testInd]
    user2game_weighted_similarity = user2game_similarity*gamesPlayed
    
    # If we just have empty arrays, then fill with 0
    if user2game_similarity.size < 1:
        user2game_similarity = 0
        user2game_weighted_similarity = 0
    
    return np.median(user2game_similarity)#, np.median(user2game_weighted_similarity)

In [ ]:
testFrame = pd.DataFrame()
testFrame['timecreated'] = userFrame.timecreated
testFrame['usercountry'] = userFrame.loccountrycode
testFrame['games']       = userFrame.games

# Initialise lists to fill dataframe
numFriends   = []
numGames     = []
playTimeList = []
gameIDs      = []
for index, row in userFrame.iterrows():
    playTime   = 0 # Default
    gamesOwned = []
    try:
        friendDict = ast.literal_eval(row['friends'])
        numFriends.append(len(friendDict))
    except:
        numFriends.append(0) # If code throws an error, it is because user has no games
    try:
        gameDict = ast.literal_eval(row['games'])
        numGames.append(len(gameDict))
        for g in gameDict:
            playTime += g['playtime_forever']/60 # divide by 60 to get time expressed in hours
            gamesOwned.append(g['appid'])
        gameIDs.append(gamesOwned)
    except:
        numGames.append(0) # If code throws an error, it is because the user has no games
        gameIDs.append(0)
    playTimeList.append(playTime)    
        

# Now we fill in the columns of the dataframe with the lists we just compted
testFrame['friends']    = numFriends
testFrame['playTime']   = playTimeList
testFrame['numGames']   = numGames
testFrame['gamesOwned'] = gameIDs
testFrame = testFrame.dropna(axis=0,how='any')

## Bring user and game data together in one frame

 Things to achieve in the cell below:
 - fill in a pandas dataframe with user information that we already have
 - into the same dataframe insert game inforamation
 - compute similarity index for tags and description

In [ ]:
testData = testFrame[:numUsers]
test     = pd.DataFrame() # initialise dataframe

for index,user in testData.iterrows():    
    # Define the users game dict that contains games owned info and hours played
    gd = ast.literal_eval(user.games)

    # This code detects which games the user owns, and generates a list of games they don't own
    # To deal with imbalance in classes, we generate a list of games that uers don't own that is 
    # the same length of the list of games that they do own. 
    gmOwn = []
    for g in gd:
        if g['appid'] in allGames:
            gmOwn.append(g['appid'])
    gamesNotOwned = [x for x in allGames if x not in gmOwn]
    #gamesNotOwned = random.choices(gamesNotOwned,k=len(gmOwn))
    
    if gmOwn: # We only want to process the user that have at least one game

        keepColumns = ['timecreated','usercountry','friends','playTime','numGames']
        userGames = gameFrame.loc[gameFrame['gameid'].isin(gmOwn)]
        userGames.loc[:,'userHasGame'] = pd.Series(np.ones(len(userGames)), index=userGames.index)
        testGames = gameFrame.loc[gameFrame['gameid'].isin(gamesNotOwned)]
        testGames.loc[:,'userHasGame'] = pd.Series(np.zeros(len(testGames)),index=testGames.index)
        tmp = pd.concat([userGames,testGames],axis=0)
        frame = pd.concat([pd.DataFrame(testData.loc[index,keepColumns]).T]*len(allGames))

        # Reindex both tmp and frame
        tmp   = tmp.reset_index()
        frame = frame.reset_index()
        userPlusGame = pd.concat([frame,tmp],axis=1)
        
       # For some strange reason I was still getting some NaN's. let's just make sure they are not included
        userPlusGame = userPlusGame.dropna(axis=0, how='any') # Remove any rows that have nans
        
        # Now we compute the td-idf similarity scores 
        userPlusGame['description_similarity'] = userPlusGame.apply(lambda row: compute_description_similarity(row['gameid'], gd, index), axis=1)
        userPlusGame['tag_similarity'] = userPlusGame.apply(lambda row: compute_tag_similarity(row['gameid'], gd, index), axis=1)
         
        # Now I can append this to the end of the dataframe
        test = test.append(userPlusGame)

        print(index,end='\r')
    # We will test for each game in our game list, so we need to duplicate the user information n*games number of times
    #test = test.append([testData.loc[[index]]]*len(userGames)*2)
test = test.reset_index()

# Process dataframe to be suitable for modeling

Our model can only process numbers, so we have to convert categorical variables (country, genre, etc) into so-called 'dummy variables' which essentially converts a list of categorical variables into a much large sparse matrix of 0's and 1's (where each column now represents each instance of a categorical variable).

In [ ]:
def preprocess_columns(df):
    # We also have to remove any weird characters from the dataframe column names
    df.columns = [col.replace(',', '') for col in df.columns]
    df.columns = [col.replace('[', '') for col in df.columns]
    df.columns = [col.replace(']', '') for col in df.columns]
    df.columns = [col.replace('<', '') for col in df.columns]
    return df

def process_frame(df):
    
    # Generate dummy dataframes from our categorical variables    
    dfCountry   = pd.get_dummies(df['usercountry'])
    dfCountry   = preprocess_columns(dfCountry)
    dfDeveloper = pd.get_dummies(df['developers'])
    dfDeveloper = preprocess_columns(dfDeveloper)
    dfGenre     = pd.get_dummies(df['genre'])
    dfGenre     = preprocess_columns(dfGenre)

    # Concatinate the dummy variables with the original dataframe
    df_new      = pd.concat([df, dfCountry, dfDeveloper, dfGenre], axis=1)
    # Remove the original categorical columns from the dataframe, keeping only the dummy variable columns
    df_new = df_new.drop(['usercountry','developers','genre','tags'],axis=1)

    # We have to convert the data type to float so that the XGBoost model can handle it
    df_new['timecreated'] = pd.to_numeric(df_new['timecreated'],downcast='float')
    df_new['friends'] = pd.to_numeric(df_new['friends'],downcast='float')
    df_new['playTime'] = pd.to_numeric(df_new['playTime'],downcast='float')
    df_new['numGames'] = pd.to_numeric(df_new['numGames'],downcast='float')
    return df_new

def add_missing_dummy_columns( d, columns ):
    # This script detects missing columns in the test dataframe, and adds them
    missing_cols = set( columns ) - set( d.columns )
    for c in missing_cols:
        d[c] = 0

def fix_columns( d, columns ):  
    # This function fixes columns in the the test dataframe to be the same as what the training dataset. 
    add_missing_dummy_columns( d, columns )

    # make sure we have all the columns we need
    assert( set( columns ) - set( d.columns ) == set())

    extra_cols = set( d.columns ) - set( columns )
    if extra_cols:
        print("extra columns: {}".format(extra_cols))

    d = d[ columns ]
    return d

# Process the dataframes by creating dummy variabels etc.
df_new = process_frame(test)

# Now we are going to load in some rows from the dataframe that was used to train the model.
testSet  = pd.read_csv('/home/trainingDataset',nrows=20)
del testSet['Unnamed: 0']
testCols = testSet.columns

# Using the above functions, we will ensure that the test dataframes are in the shape the model is expecting
df_new = fix_columns(df_new,testCols)

In [ ]:
# Make a list of feature headers
featureHeaders = list(df_new.columns)
targetFeature = 'userHasGame'
featureHeaders.remove(targetFeature)
mkList = []
for x in featureHeaders:
    mkList.append(x)   

## Load in previously trained model

In [ ]:
modelList = pickle.load(open('/home/ensemble_model.sav', 'rb'))

## Loop through models and compute predictions

In [ ]:
pred  = np.zeros((len(modelList),len(df_new)))
prob  = np.zeros((len(modelList),len(df_new)))
for ind,model in enumerate(modelList):
    pred[ind,:] = model.predict(df_new[mkList])
    tmp = model.predict_proba(df_new[mkList])
    prob[ind,:] = tmp[:,1]

## Compute model performance metrics

In [ ]:
predictions = np.round(np.median(pred,axis=0))

Accuracy = accuracy_score(df_new[targetFeature], predictions)
rc = recall_score(df_new[targetFeature],predictions)
pc = precision_score(df_new[targetFeature],predictions)
print("Accuracy = {}".format(np.round(Accuracy,2)))
print("Precision = {}".format(np.round(pc,2)))
print("Recall = {}".format(np.round(rc,2)))

## Define function for plotting the confusion matrix

A confsusion matrix is a well known plot that can be used for assessing the performance of a classification model by summarizing how well the model correctly assigns classes. 

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap,vmin=0,vmax=1)
    plt.title(title,fontsize=15)
    plt.colorbar(shrink=0.7)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45,fontsize=12)
    plt.yticks(tick_marks, classes,rotation=45,fontsize=12)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), fontsize=15,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label',fontsize=15)
    plt.xlabel('Predicted label',fontsize=15)

## Plot the confusion matrix of the tuned model

In [ ]:
cm = confusion_matrix(df_new[targetFeature],predictions)
plt.figure(figsize=(5,5),dpi=100)
plot_confusion_matrix(cm, classes=['User owns game','User does not'], normalize=True,
                      title='Normalized confusion matrix')
plt.show()

## Compute ROC curve 

The ROC curve, or receive operating characteristic, is a great measure for quickly assessing the performance of a classification model. This plot shows how the model specificity (ability to detect classes) changes as a function of the threshold for classification. By plotting like this, we can observe the tradeoff between false positive rate and true positive rate. 

In [ ]:
probabilities = np.mean(prob,axis=0)
fpr, tpr, _ = metrics.roc_curve(df_new[targetFeature], probabilities)
roc_auc = metrics.auc(fpr, tpr)

plt.figure(figsize=(6, 5),dpi=100)
plt.plot(fpr, tpr, color='navy',
             lw=3, label='ROC curve (area = {})'.format(round(roc_auc,2)))
plt.plot([0, 1], [0, 1], color='darkred', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate',fontsize=15)
plt.ylabel('True Positive Rate',fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(loc="lower right",fontsize=15)
plt.show()